<a href="https://colab.research.google.com/github/TayJen/hackathon_algu2022/blob/master/H_ALGU_2022_Advance%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
%cd /content/drive/Shareddrives/data_drive/H_Vladivostok2022
!pwd

/content/drive/Shareddrives/data_drive/H_Vladivostok2022
/content/drive/Shareddrives/data_drive/H_Vladivostok2022


## Импорт библиотек

In [28]:
# Для работы с данными
import numpy as np
import pandas as pd

# Для моделей
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor

# Метрика
from sklearn.metrics import r2_score

%matplotlib inline
np.random.seed(59)

# Проверка гипотез, работа с данными

## Взгляд на данные

In [29]:
df = pd.read_csv('./data/prepared/train.csv')
df.head()

,project_id,assignee_id,creator_id,month,day,hour,minute,key_name,key_num,lemm_summary,...,hiring_type,payment_type,passport,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,text,comments_cnt,authors_cnt
0,5,93,93,10,1,5,57,SM,10678,ui тест по заказ добро кейджи,...,staff,fixed,1,1,1,1,1,NaN,0,0
1,5,93,93,10,1,5,59,SM,10679,ui тест раздел профиль,...,staff,fixed,1,1,1,1,1,приверять и принимать mr,1,1
2,5,93,93,10,1,6,0,SM,10680,ui тест раздел личный счет,...,staff,fixed,1,1,1,1,1,NaN,0,0
3,5,93,93,10,1,6,2,SM,10682,ui тест раздел новость,...,staff,fixed,1,1,1,1,1,NaN,0,0
4,5,93,93,10,1,6,3,SM,10683,ui тест раздел зона скидка и доплата,...,staff,fixed,1,1,1,1,1,принимать mr,1,1


In [30]:
df_test = pd.read_csv('./data/prepared/test.csv')
df_test.head()

,project_id,assignee_id,creator_id,month,day,hour,minute,key_name,key_num,lemm_summary,...,hiring_type,payment_type,passport,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,text,comments_cnt,authors_cnt
0,5,112,112,1,15,9,52,SM,11140,выгружать весь mp файл звонок на диск,...,staff,fixed,0,0,0,0,0,ресерч по rsync http losst ru rsync primery si...,8,1
1,5,112,112,1,15,9,54,SM,11141,удалять файл mp звонок с сервер до,...,staff,fixed,0,0,0,0,0,задача приостанавливать в связь с заморозка ра...,4,2
2,5,94,54,1,16,4,46,SM,11144,удалять окно выбор тариф и перенести функциона...,...,staff,fixed,0,0,0,0,0,задача закрывать в связь с передача проект,1,1
3,5,94,94,1,16,11,12,SM,11146,при авторизация пользователь водитель не видет...,...,staff,fixed,0,0,0,0,0,не правильно реализовать логика вызов метод ge...,2,2
4,5,1,54,1,17,5,19,SM,11150,основной проблема по корп отдел по состояние н...,...,unknown,unknown,0,0,0,0,0,ch keneskhanova прежде все по проблема коррект...,7,2


In [31]:
df['text'] = df['text'].fillna('')
df_test['text'] = df_test['text'].fillna('')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9589 entries, 0 to 9588
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   project_id                  9589 non-null   int64  
 1   assignee_id                 9589 non-null   int64  
 2   creator_id                  9589 non-null   int64  
 3   month                       9589 non-null   int64  
 4   day                         9589 non-null   int64  
 5   hour                        9589 non-null   int64  
 6   minute                      9589 non-null   int64  
 7   key_name                    9589 non-null   object 
 8   key_num                     9589 non-null   int64  
 9   lemm_summary                9589 non-null   object 
 10  log_target                  9589 non-null   float64
 11  active                      9589 non-null   int64  
 12  position                    9589 non-null   object 
 13  hiring_type                 9589 

## Общий tf-idf

Попробуем выполнить общий tf-idf, с ограничением в N токенов, как для описания задачи, так и для комментариев

In [32]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stopwords_rus = stopwords.words("russian")
stopwords_eng = stopwords.words('english')
stopwords_all = stopwords_rus + stopwords_eng

summ_tf_idf = TfidfVectorizer(stop_words=stopwords_all, max_features=7000)
comm_tf_idf = TfidfVectorizer(stop_words=stopwords_all, max_features=3000)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
len_train, len_test = len(df), len(df_test)
len_train, len_test

(9589, 1070)

In [34]:
df_general = pd.concat([df, df_test], axis=0, ignore_index=True)
df_general.shape

(10659, 23)

In [35]:
X_summ_new = summ_tf_idf.fit_transform(df_general['lemm_summary'])
feature_names_summ = list(map(lambda x: x + '_summ', summ_tf_idf.get_feature_names_out()))

X_summ_new_df = pd.DataFrame(X_summ_new.toarray(), columns=feature_names_summ)
df_general.drop('lemm_summary', axis=1, inplace=True)
df_general = pd.concat([df_general, X_summ_new_df], axis=1)

df_general.shape

(10659, 5056)

In [36]:
# X_comm_new = comm_tf_idf.fit_transform(df_general['text'])
# feature_names_comm = list(map(lambda x: x + '_comm', comm_tf_idf.get_feature_names_out()))

# X_comm_new_df = pd.DataFrame(X_comm_new.toarray(), columns=feature_names_comm)
df_general.drop('text', axis=1, inplace=True)
# df_general = pd.concat([df_general, X_comm_new_df], axis=1)

# df_general.shape

## Преобразование категориальных признаков

In [37]:
df.key_name.unique(), df_test.key_name.unique()

(array(['SM', 'BALT', 'BA', 'UN', 'FPY', 'XXO', 'ADBK', 'ADBKRK', 'LJ'],
       dtype=object),
 array(['SM', 'BALT', 'BAD', 'UN', 'FPY', 'XXO', 'ADBK', 'ADBKRK', 'LJ'],
       dtype=object))

In [38]:
df_general['key_name'] = df_general['key_name'].apply(lambda x: 'BA' if x == 'BAD' else x)
df_general.key_name.unique()

array(['SM', 'BALT', 'BA', 'UN', 'FPY', 'XXO', 'ADBK', 'ADBKRK', 'LJ'],
      dtype=object)

In [39]:
cat_features = ['key_name', 'position', 'hiring_type', 'payment_type']

ord_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',
                             unknown_value=-1)

df_general[cat_features] = ord_encoder.fit_transform(df_general[cat_features])

## Разделение данных

In [40]:
df_train = df_general.loc[:len_train - 1].copy()
df_test = df_general.loc[len_train:].copy()
df_train.shape, df_test.shape

((9589, 5055), (1070, 5055))

In [41]:
df_train.head()

,project_id,assignee_id,creator_id,month,day,hour,minute,key_name,key_num,log_target,...,энергосбережение_summ,эпик_summ,это_summ,эффект_summ,юзер_summ,юнит_summ,ядро_summ,язык_summ,январь_summ,ящик_summ
0,5,93,93,10,1,5,57,6.0,10678,7.495542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,93,93,10,1,5,59,6.0,10679,8.881836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,93,93,10,1,6,0,6.0,10680,9.574983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,93,93,10,1,6,2,6.0,10682,6.802395,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,93,93,10,1,6,3,6.0,10683,6.802395,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df_train.tail()

,project_id,assignee_id,creator_id,month,day,hour,minute,key_name,key_num,log_target,...,энергосбережение_summ,эпик_summ,это_summ,эффект_summ,юзер_summ,юнит_summ,ядро_summ,язык_summ,январь_summ,ящик_summ
9584,48,1,116,6,15,9,47,0.0,779,10.167389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9585,56,2,2,6,2,10,8,5.0,1,9.692767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9586,56,2,2,6,5,9,52,5.0,3,8.188689,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9587,56,2,2,6,5,13,46,5.0,4,8.748305,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9588,56,2,2,6,9,2,56,5.0,5,8.594154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df_test.drop(['log_target'], axis=1, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_test.head()

,project_id,assignee_id,creator_id,month,day,hour,minute,key_name,key_num,active,...,энергосбережение_summ,эпик_summ,это_summ,эффект_summ,юзер_summ,юнит_summ,ядро_summ,язык_summ,январь_summ,ящик_summ
0,5,112,112,1,15,9,52,6.0,11140,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,5,112,112,1,15,9,54,6.0,11141,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,5,94,54,1,16,4,46,6.0,11144,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,5,94,94,1,16,11,12,6.0,11146,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,5,1,54,1,17,5,19,6.0,11150,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.447199,0.0


# Работа с моделью

In [44]:
from sklearn.model_selection import KFold
from tqdm import tqdm

X = df_train.drop(['log_target'], axis=1).values
y = df_train[['log_target']].values

In [45]:
scores = []
kf = KFold(n_splits=5, shuffle=True, random_state=59)
for i, (train_index, val_index) in tqdm(enumerate(kf.split(X))):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = XGBRegressor(max_depth=21, random_state=59,
                         objective='reg:squarederror',
                         tree_method='gpu_hist')
    model.fit(X_train, y_train)

    pred = model.predict(X_val)
    score = r2_score(y_val, pred)
    print(f' Fold {i}, score: {score}')
    scores.append(score)

print(np.mean(scores))

1it [00:41, 41.33s/it]

 Fold 0, score: 0.2781515791779


2it [01:20, 40.32s/it]

 Fold 1, score: 0.27992968474331525


3it [01:59, 39.58s/it]

 Fold 2, score: 0.2929717263365228


4it [02:39, 39.84s/it]

 Fold 3, score: 0.24544126824014822


5it [03:20, 40.19s/it]

 Fold 4, score: 0.266092330708396
0.27251731784125643


In [46]:
model = XGBRegressor(max_depth=21, random_state=59,
                     objective='reg:squarederror',
                     tree_method='gpu_hist')
model.fit(X, y)

XGBRegressor(max_depth=21, objective='reg:squarederror', random_state=59,
             tree_method='gpu_hist')

## Предсказание на сайт

In [48]:
X_test = df_test.values
prediction = model.predict(X_test)
prediction = np.exp(prediction)
prediction = np.rint(prediction).astype('int64')
prediction

array([ 9800, 11485,  6724, ...,  5808,  5726,  8650])

In [49]:
df_sample = pd.read_csv('./data/sample_solution.csv')
df_sample.overall_worklogs = prediction
df_sample.head()

,id,overall_worklogs
0,675975,9800
1,675972,11485
2,675965,6724
3,675961,6812
4,675955,5807


In [50]:
df_sample.to_csv('./solutions/d21_5000_no_comms.csv', index=False)

## Проверка предсказания

In [51]:
df_solution = pd.read_csv('./solutions/d21_5000_no_comms.csv')
df_solution.head()

,id,overall_worklogs
0,675975,9800
1,675972,11485
2,675965,6724
3,675961,6812
4,675955,5807
